<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Working with a real world data-set using SQL and Python

Estaimted time needed: **30** minutes

Name: **Max Emanuel Feucht**


## Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTE**: 

Do not download the dataset directly from City of Chicago portal. Instead download a static copy which is a more database friendly version from this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_edX/data/Chicago_Public_Schools_-_Progress_Report_Cards__2011-2012-v3.csv">link</a>.

Now review some of its contents.


### Connect to the database

Let us now load the ipython-sql  extension and establish a connection with the database


In [1]:
%%sh
wget https://raw.githubusercontent.com/DB2-Samples/Db2re/master/db2re.ipynb

--2020-10-10 19:15:38--  https://raw.githubusercontent.com/DB2-Samples/Db2re/master/db2re.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84942 (83K) [text/plain]
Saving to: ‘db2re.ipynb.2’

     0K .......... .......... .......... .......... .......... 60% 18.2M 0s
    50K .......... .......... .......... ..                   100% 20.6M=0.004s

2020-10-10 19:15:38 (19.1 MB/s) - ‘db2re.ipynb.2’ saved [84942/84942]



In [2]:
!pip install --user qgrid

In [3]:
%run db2re.ipynb

Pixiedust database opened successfully


DB2 Extensions Loaded.


In [4]:
!pip install ipython-sql

In [8]:
%load_ext sql 

In [12]:
%sql ibm_db_sa://ppz81225:v1thndhp0915f-49@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB

### Query the database system catalog to retrieve table metadata


In [19]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)

%sql select TABSCHEMA, TABNAME, CREATE_TIME from syscat.tables\
where TABSCHEMA = 'PPZ81225'


 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,create_time
PPZ81225,EMPLOYEES,2020-10-09 06:50:46.650530
PPZ81225,JOB_HISTORY,2020-10-09 06:50:46.818987
PPZ81225,LOCATIONS,2020-10-09 06:50:47.272549
PPZ81225,JOBS,2020-10-09 07:15:42.989211
PPZ81225,DEPARTMENTS,2020-10-09 07:26:27.849631
PPZ81225,PETSALE,2020-10-09 13:31:32.747647
PPZ81225,INSTRUCTOR,2020-10-10 10:38:19.104707
PPZ81225,INTERNATIONAL_STUDENT_TEST_SCORES,2020-10-10 11:45:15.206576
PPZ81225,CHICAGO_SOCIOECONOMIC_DATA,2020-10-10 12:29:32.949000
PPZ81225,CSD,2020-10-10 12:30:39.409570


### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?


In [21]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql select count(*) from syscat.columns\
where TABNAME = 'CHICAGO'

 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


1
78


Now retrieve  the list of columns in SCHOOLS table and their column type (datatype) and length.


In [23]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql select COLNAME, TYPENAME, LENGTH from syscat.columns\
where TABNAME = 'CHICAGO' 

 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


colname,typename,length
School_ID,INTEGER,4
NAME_OF_SCHOOL,VARCHAR,65
"Elementary, Middle, or High School",VARCHAR,2
Street_Address,VARCHAR,30
City,VARCHAR,7
State,VARCHAR,2
ZIP_Code,INTEGER,4
Phone_Number,VARCHAR,14
Link,VARCHAR,78
Network_Manager,VARCHAR,40


### Questions

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "\_"?


In [27]:
%sql select * from CHICAGO limit 5;

 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


School_ID,name_of_school,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,Collaborative_Name,Adequate_Yearly_Progress_Made_,Track_Schedule,CPS_Performance_Policy_Status,CPS_Performance_Policy_Level,healthy_school_certified,Safety_Icon,safety_score,Family_Involvement_Icon,Family_Involvement_Score,Environment_Icon,Environment_Score,Instruction_Icon,Instruction_Score,Leaders_Icon,Leaders_Score,Teachers_Icon,Teachers_Score,Parent_Engagement_Icon,Parent_Engagement_Score,Parent_Environment_Icon,Parent_Environment_Score,average_student_attendance,Rate_of_Misconducts__per_100_students_,Average_Teacher_Attendance,Individualized_Education_Program_Compliance_Rate,Pk_2_Literacy__,Pk_2_Math__,Gr3_5_Grade_Level_Math__,Gr3_5_Grade_Level_Read__,Gr3_5_Keep_Pace_Read__,Gr3_5_Keep_Pace_Math__,Gr6_8_Grade_Level_Math__,Gr6_8_Grade_Level_Read__,Gr6_8_Keep_Pace_Math_,Gr6_8_Keep_Pace_Read__,Gr_8_Explore_Math__,Gr_8_Explore_Read__,ISAT_Exceeding_Math__,ISAT_Exceeding_Reading__,ISAT_Value_Add_Math,ISAT_Value_Add_Read,ISAT_Value_Add_Color_Math,ISAT_Value_Add_Color_Read,Students_Taking__Algebra__,Students_Passing__Algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),Net_Change_EXPLORE_and_PLAN,11th Grade Average ACT (2011),Net_Change_PLAN_and_ACT,College_Eligibility__,Graduation_Rate__,College_Enrollment_Rate__,college_enrollment,General_Services_Route,Freshman_on_Track_Rate__,x_coordinate,y_coordinate,Latitude,Longitude,community_area_number,community_area_name,Ward,Police_District,Location
610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf,Fullerton Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Very Strong,99,Very Strong,99,Strong,74,Strong,66,Strong,65,Strong,70,Strong,56,Average,47,96.00%,2.0,96.40%,95.80%,80.1,43.3,89.6,84.9,60.7,62.6,81.9,85.2,52,62.4,66.3,77.9,69.7,64.4,0.2,0.9,Yellow,Green,67.1,54.5,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,813,33,NDA,1171699.458,1915829.428,41.92449696,-87.64452163,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
610281,Adam Clayton Powell Paideia Community Academy Elementary School,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610281.pdf,Skyway Elementary Network,SOUTH SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,54,Strong,66,Strong,74,Very Strong,84,Strong,63,Strong,76,Weak,46,Average,50,95.60%,15.7,95.30%,100.00%,62.4,51.7,21.9,15.1,29,42.8,38.5,27.4,44.8,42.7,14.1,34.4,16.8,16.5,0.7,1.4,Green,Green,17.2,27.3,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,521,46,NDA,1196129.985,1856209.466,41.76032435,-87.55673627,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610185.pdf,Midway Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 2,No,Strong,61,NDA,NDA,Average,50,Weak,36,NDA,NDA,NDA,NDA,Average,47,Weak,41,95.70%,2.3,94.70%,98.30%,53.7,26.6,38.3,34.7,43.7,57.3,48.8,39.2,46.8,44,7.5,21.9,18.3,15.5,-0.9,-1.0,Red,Red,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,1324,44,NDA,1148427.165,1851012.215,41.74711093,-87.73170248,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609993.pdf,Pershing Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,56,Average,44,Average,45,Weak,37,Strong,65,Average,48,Average,53,Strong,58,95.50%,10.4,95.80%,100.00%,76.9,NDA,26,24.7,61.8,49.7,39.2,27.2,69.7,60.6,9.1,18.2,11.1,9.6,0.9,2.4,Green,Green,42.9,25,NDA,NDA,NDA,NDA,NDA,NDA,

## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?


In [31]:
%sql select count(*) from CHICAGO \
where "Elementary, Middle, or High School" = 'ES';

 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


1
462


### Problem 2

##### What is the highest Safety Score?


In [32]:
%sql select max(safety_score) from CHICAGO;

 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


1
99


### Problem 3

##### Which schools have highest Safety Score?


In [35]:
%sql select name_of_school from CHICAGO \
where safety_score = (select max(safety_score) from CHICAGO);


 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school
Abraham Lincoln Elementary School
Alexander Graham Bell Elementary School
Annie Keller Elementary Gifted Magnet School
Augustus H Burley Elementary School
Edgar Allan Poe Elementary Classical School
Edgebrook Elementary School
Ellen Mitchell Elementary School
James E McDade Elementary Classical School
James G Blaine Elementary School
LaSalle Elementary Language Academy


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [39]:
%sql select name_of_school from CHICAGO \
order by average_student_attendance desc \
limit 10;

 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school
Velma F Thomas Early Childhood Center
John Charles Haines Elementary School
James Ward Elementary School
Edgar Allan Poe Elementary Classical School
Rachel Carson Elementary School
Orozco Fine Arts & Sciences Elementary School
Annie Keller Elementary Gifted Magnet School
Andrew Jackson Elementary Language Academy
Lenart Elementary Regional Gifted Center
Disney II Magnet School


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance


In [43]:
%sql select name_of_school, average_student_attendance from CHICAGO \
order by average_student_attendance \
limit 5;

 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,average_student_attendance
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%
Orr Academy High School,66.30%


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column


In [45]:
%sql select name_of_school, replace(average_student_attendance, '%', '')from CHICAGO \
order by average_student_attendance \
limit 5;

 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,2
Richard T Crane Technical Preparatory High School,57.90
Barbara Vick Early Childhood & Family Center,60.90
Dyett High School,62.50
Wendell Phillips Academy High School,63.00
Orr Academy High School,66.30


**Notiz: replace(average_student_attendance, '%', '') zum Löschen von Zeichen aus einem Result Set**

### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?


In [53]:
%sql select name_of_school from CHICAGO \
where decimal(replace(average_student_attendance, '%','')) < 70.00 ;
              


 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school
Barbara Vick Early Childhood & Family Center
Chicago Vocational Career Academy High School
Dyett High School
Manley Career Academy High School
Orr Academy High School
Richard T Crane Technical Preparatory High School
Roberto Clemente Community Academy High School
Wendell Phillips Academy High School


### Problem 8

##### Get the total College Enrollment for each Community Area


In [64]:
%sql select COMMUNITY_AREA_NAME, sum(COLLEGE_ENROLLMENT) as TOTAL_ENROLLMENT from CHICAGO \
group by COMMUNITY_AREA_NAME; 

 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,total_enrollment
ALBANY PARK,6864
ARCHER HEIGHTS,4823
ARMOUR SQUARE,1458
ASHBURN,6483
AUBURN GRESHAM,4175
AUSTIN,10933
AVALON PARK,1522
AVONDALE,3640
BELMONT CRAGIN,14386
BEVERLY,1636


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order


In [164]:
%sql select COMMUNITY_AREA_NAME, sum(COLLEGE_ENROLLMENT) as TOTAL_ENROLLMENT from CHICAGO \
group by COMMUNITY_AREA_NAME \
order by sum(COLLEGE_ENROLLMENT) desc\
limit 5; 

 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,total_enrollment
SOUTH LAWNDALE,14793
BELMONT CRAGIN,14386
AUSTIN,10933
GAGE PARK,9915
BRIGHTON PARK,9647


### Problem 10

##### Get the hardship index for the community area which has College Enrollment of 4368


In [78]:
%sql select hardship_index from csd CD, CHICAGO C \
where CD.ca = C.community_area_number\
and college_enrollment = 4368;

 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


hardship_index
6.0


### Problem 11

##### Get the hardship index for the community area which has the highest value for College Enrollment


In [181]:
%sql select CD.hardship_index, C.community_area_name from csd CD, CHICAGO C \
where CD.ca = C.community_area_number\
and C.college_enrollment = (select max(college_enrollment) from CHICAGO);

 * ibm_db_sa://ppz81225:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


hardship_index,community_area_name
6.0,NORTH CENTER


## Summary

##### In this lab you demonstrated how to work with a real word dataset using SQL and Python. You showed how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.


## Teacher

<a href="https://www.linkedin.com/in/ravahuja/" target="_blank">Rav Ahuja</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2020-08-28        | 2.0     | Lavanya    | Moved lab to course repo in GitLab |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
